<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/VoiD_with_PySpark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Installing SPARK for Playing


In [23]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!pip install -q findspark
import findspark
findspark.init()


# SPARK Installed... Start playing

In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [26]:
!wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
!ls
!wget -q "https://data.dws.informatik.uni-mannheim.de/structureddata/2021-12/quads/classspecific/schema_Mountain/part_0.gz"
!ls

part_0.gz    sample_data	  small_page_links.nt.1
part_0.gz.1  small_page_links.nt  small_page_links.nt.2
part_0.gz    part_0.gz.2  small_page_links.nt	 small_page_links.nt.2
part_0.gz.1  sample_data  small_page_links.nt.1


In [27]:
lines = spark.read.text("part_0.gz").rdd.map(lambda r: r[0])



In [28]:
lines.take(5)

['_:nb53a24408607424384c1357880ce1bc7xb0 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Mountain> <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb1 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb2 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb3 <https://peakery.com/dragontail-peak-washington/>   .',
 '_:nb53a24408607424384c1357880ce1bc7xb0 <http://schema.org/additionalProperty> _:nb53a24408607424384c1357880ce1bc7xb4 <https://peakery.com/dragontail-peak-washington/>   .']

In [29]:
lines.count()

2113514

In [30]:
import re
def parseTriple(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    return parts[0], parts[1], parts[2],parts[3]
triples=lines.map(lambda x:parseTriple(x))
print(triples.count())
triples.take(1)

2113514


[('_:nb53a24408607424384c1357880ce1bc7xb0',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://schema.org/Mountain>',
  '<https://peakery.com/dragontail-peak-washington/>')]

# Exercice: Computing the VoiD for the Nt File with PySpark RDD

Write RDD PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

- QB1 CONSTRUCT { <D> v:triples ?x }
WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

- QB2 CONSTRUCT { <D> v:classes ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

- QB3 CONSTRUCT { <D> v:properties ?x }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

- QB4 CONSTRUCT { <D> v:distinctSubjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

- QB5 CONSTRUCT { <D> v:distinctObjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [31]:
print(triples.map(lambda x: x[0]).distinct().count())
print(triples.map(lambda x: x[1]).distinct().count())
print(triples.map(lambda x: x[2]).distinct().count())
print(triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>").map(lambda x: x[2]).distinct().count())


487829
253
829323
304


- QC1 CONSTRUCT { <D> v:classPartition [ v:class ?c ] } WHERE { ?s a ?c }

- QC2 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:triples ?x ] }
WHERE { SELECT (COUNT(?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }
- QC3 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:classes ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?d) AS ?x) ?c WHERE { ?s a ?c , ?d } GROUP BY ?c }
- QC4 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:properties ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c } 
- QC5 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c WHERE { ?s a ?c } GROUP BY ?c }
- QC6 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }

READ the DOC !! https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations 

In [32]:
## testing operators 
## hmm join consider only pairs !!
sc=spark.sparkContext
x = sc.parallelize([("a", 1,"x"), ("b", 4,"y")])
y = sc.parallelize([("a", 2,"z"), ("a", 3,"w")])
sorted(x.join(y).collect())


[('a', (1, 2)), ('a', (1, 3))]

In [33]:
## keyby
print(sc.parallelize(range(0,3)).collect())
sc.parallelize(range(0,3)).keyBy(lambda x: x*x).collect()

[0, 1, 2]


[(0, 0), (1, 1), (4, 2)]

In [42]:
# testing simple GC2 on simple data

#simple data
data=sc.parallelize([("s1","a","prof"),("s1","name","pascal"),("s2","a","prof"),("s2","name","hala"),("s3","a","student"),("s3","name","tom")])

# 
# select count(?p) where {
#    ?s a ?c . #tp1
#    ?s ?p ?o #tp2
#} groupby ?c

tp1=data.filter(lambda x: x[1]=="a").keyBy(lambda x: x[0])
print("tp1:",tp1.collect())
tp2=data.keyBy(lambda x:x[0])
print("tp2:",tp2.collect())
bgp=tp1.join(tp2).values()
print("tp1 . tp2:",bgp.mapValues(list).collect())
bgp.mapValues(list).collect()


groupby=bgp.keyBy(lambda x: x[0][2]).groupByKey()
groupby.mapValues(list).collect()
#print("goupby ?c:",groupby.mapValues(list).collect())

#keep only unique predicate in the bag
# list(dict.fromkeys -> eliminate duplicate in a python list
gbp=groupby.mapValues(lambda x: list(dict.fromkeys(list(map(lambda bag: bag[1][1],x)))))
#print("goupby ?c projection:",gbp.collect())
gbp.collect()

# remember avoid collect on real execution...
print("final:",gbp.mapValues(lambda x: len(x)).collect())





tp1: [('s1', ('s1', 'a', 'prof')), ('s2', ('s2', 'a', 'prof')), ('s3', ('s3', 'a', 'student'))]
tp2: [('s1', ('s1', 'a', 'prof')), ('s1', ('s1', 'name', 'pascal')), ('s2', ('s2', 'a', 'prof')), ('s2', ('s2', 'name', 'hala')), ('s3', ('s3', 'a', 'student')), ('s3', ('s3', 'name', 'tom'))]
tp1 . tp2: [(('s1', 'a', 'prof'), ['s1', 'a', 'prof']), (('s1', 'a', 'prof'), ['s1', 'name', 'pascal']), (('s3', 'a', 'student'), ['s3', 'a', 'student']), (('s3', 'a', 'student'), ['s3', 'name', 'tom']), (('s2', 'a', 'prof'), ['s2', 'a', 'prof']), (('s2', 'a', 'prof'), ['s2', 'name', 'hala'])]
final: [('student', 2), ('prof', 2)]


In [49]:
triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" and  x[2]=="<https://schema.org/Mountain>").distinct().count()


22258

Montagne et plage...

In [55]:
tp1=triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" and x[2]=="<https://schema.org/Mountain>").keyBy(lambda x: x[3])
tp2=triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" and x[2]=="<https://schema.org/Beach>").keyBy(lambda x: x[3])
joined=tp2.join(tp1)
joined.take(5)


[('<https://mapcarta.com/fr/17320490>',
  (('_:node1flttgbvhx28909215',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909182',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Mountain>',
    '<https://mapcarta.com/fr/17320490>'))),
 ('<https://mapcarta.com/fr/17320490>',
  (('_:node1flttgbvhx28909215',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909222',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Mountain>',
    '<https://mapcarta.com/fr/17320490>'))),
 ('<https://mapcarta.com/fr/17320490>',
  (('_:node1flttgbvhx28909215',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909223',
    '<ht

In [70]:
# bs = sujet des Beach
bs=joined.values().map(lambda x : x[0]).keyBy(lambda x : x[0])
name=triples.filter(lambda x: x[1]=="<https://schema.org/name>").keyBy(lambda x: x[0])
bs.join(name).collect()

[('_:node1flttgbvhx28909217',
  (('_:node1flttgbvhx28909217',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909217',
    '<https://schema.org/name>',
    '"Plage',
    'Ait'))),
 ('_:node1flttgbvhx28909217',
  (('_:node1flttgbvhx28909217',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909217',
    '<https://schema.org/name>',
    '"Plage',
    'Ait'))),
 ('_:node1flttgbvhx28909217',
  (('_:node1flttgbvhx28909217',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://schema.org/Beach>',
    '<https://mapcarta.com/fr/17320490>'),
   ('_:node1flttgbvhx28909217',
    '<https://schema.org/name>',
    '"Plage',
    'Ait'))),
 ('_:node1flttgbvhx28909217',
  (('_:node1flttgbvhx28909217',
    '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
    '<https://

In [35]:
#QC2 for real now...

tp1=triples.filter(lambda x: x[1]=="<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>").keyBy(lambda x: x[0])
tp2=triples.keyBy(lambda x:x[0])
bgp=tp1.join(tp2).values()

groupby=bgp.keyBy(lambda x: x[0][2]).groupByKey()

#keep only unique predicate in the bag
gbp=groupby.mapValues(lambda x: list(dict.fromkeys(list(map(lambda bag: bag[1][1],x)))))

# remember avoid collect on real execution...
gbp.mapValues(lambda x: len(x)).collect()


[('<http://schema.org/CatholicChurch>', 3),
 ('<http://schema.org/NewsArticle>', 11),
 ('<http://schema.org/TireShop>', 8),
 ('<https://schema.org/Airport>', 11),
 ('<http://dbpedia.org/ontology/Location>', 50),
 ('<https://schema.org/ShoppingCenter>', 10),
 ('<http://schema.org/SearchAction>', 3),
 ('<https://schema.org/ConvenienceStore>', 11),
 ('<https://schema.org/Zoo>', 5),
 ('<https://schema.org/IceCreamShop>', 3),
 ('<https://schema.org/PoliceStation>', 11),
 ('<https://schema.org/HardwareStore>', 10),
 ('<http://schema.org/CafeOrCoffeeShop>', 9),
 ('<http://schema.org/Map>', 2),
 ('<http://www.w3.org/2002/07/owl#Thing>', 50),
 ('<https://schema.org/AdministrativeArea>', 12),
 ('<http://schema.org/ListItem>', 7),
 ('<https://schema.org/BarOrPub>', 10),
 ('<http://schema.org/PoliceStation>', 8),
 ('<http://schema.org/GeoCoordinates>', 7),
 ('<https://schema.org/LakeBodyOfWater>', 11),
 ('<https://schema.org/FireStation>', 10),
 ('<http://schema.org/LocationFeatureSpecification>',

- QD1 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ] } WHERE { ?s ?p ?o }
- QD2 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:triples ?x ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD3 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD4 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }

In [36]:
Your code ??

SyntaxError: ignored

- QE1 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:propertyPartition [ v:property ?p ] ] }
WHERE { ?s a ?c ; ?p ?o }
- QE2 CONSTRUCT { <D> v:classPartition [ v:class ?c
v:propertyPartition [ v:property ?p ; v:triples ?x ] ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE3 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctSubjects ?x ] ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE4 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctObjects ?x ; v:property ?p ] ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }

In [ ]:
Your code

# Same Exercice with PySpark SQL



In [ ]:
from pyspark.sql import Row
spo=triples.map(lambda x: Row(subject=x[0],predicate=x[1],object=x[2]))
df=spark.createDataFrame(spo)
df.createOrReplaceTempView("SPO")
df.show()



In [ ]:

result=spark.sql("select subject from SPO")
result.show()

Write SQL PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

QB1 CONSTRUCT { v:triples ?x } WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

QB2 CONSTRUCT { v:classes ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

QB3 CONSTRUCT { v:properties ?x } WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

QB4 CONSTRUCT { v:distinctSubjects ?x } WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

QB5 CONSTRUCT { v:distinctObjects ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [ ]:
Your Code